In [1]:
import pandas as pd
pd.options.display.max_rows = 999
import numpy as np
import time
import datetime as dt
import os
import re
import sys
import string
from sklearn import preprocessing
#sklearn
# from sklearn.cross_validation import train_test_split
from sklearn import metrics

from custom_transformer import ColumnSelector, ApplyFunction, ColumnTransform, PrintThat
import custom_transformer as ct

In [2]:
dataset = pd.read_csv("ticket_all_lev.csv")
# dataset = dataset.sort_values(["tpl_id"],ascending=1)
# dataset = dataset[(dataset['tpl_id']==40.0)]
ds_clean = pd.DataFrame(columns=["ticket_msg","ticket_title","tpl_id"])
ds_clean["ticket_msg"] = dataset["ticket_msg"]
ds_clean["ticket_title"] = dataset["ticket_title"]

In [3]:
for i in range (0,len(dataset["tpl_id"])):
    if pd.notnull(dataset["tpl_id"][i]):
        ds_clean["tpl_id"][i] = dataset["tpl_id"][i]
    else :
        ds_clean["tpl_id"][i] = dataset["highest_tpl_id"][i]
#     print(ds_clean["tpl_id"][i])

ds_clean["tpl_id"] = ds_clean["tpl_id"].astype('int')

In [ ]:
def coba(x,array):
    x=str(x.lower())
    words = x.split()
#     print (words)
    words = [x for x in words if x in array]
#     print (words)
    return(" ".join(words).lower())

In [ ]:
c= coba(ds_clean["ticket_msg"][0],list(stop_words["word"]))

In [ ]:
stopsList = pd.read_csv("stopwordID.csv")
# stops = list(stopsList["word"].tail(15000))
# stops = stopsList[(stopsList["count"]>=50)]
# stops = list(stops["word"])
stopsList

In [ ]:
word=ds_clean["ticket_msg"][0]
word=word.lower()
word=word.split()
print (word)
# my_list = filter(lambda word: word in stop_words, word)
my_list = [x for x in word if x =="saya"]

In [ ]:
my_list

In [ ]:
# wordstring = 'it was the best of times it was the worst of times '
# wordstring += 'it was the age of wisdom it was the age of foolishness'


def leastOccur(array):
    wordfreq = []
    wordlist = []
    allwordlist = []
    for s in (array):
        s=str(s)
#         print (s)
        wordListTemp = s.split()
        for w in wordListTemp:
            if ((w[:3] != 'inv') and (len(w)!=2)):
                print (w)
                w = ct.clean_symbol(w)
                if (w):
                    allwordlist.append(w.lower())
                    if (w not in wordlist) and (not w.isdigit()):
                        wordlist.append(w.lower())
    
    for w in wordlist :
        wordfreq.append(allwordlist.count(w))
        
    return wordlist,wordfreq

wordList = pd.DataFrame(columns = ["word","count"])
wordList["word"],wordList["count"] = leastOccur(ds_clean["ticket_title"])

print (len(wordList))
# print "String\n" + wordstring +"\n"
# print "List\n" + str(wordlist) + "\n"
# print "Frequencies\n" + str(wordfreq) + "\n"
# print "Pairs\n" + str(zip(wordlist, wordfreq))

stop_words = (wordList.sort_values(["count"],ascending=0)).head(200)

In [ ]:
(wordList.sort_values(["count"],ascending=0)).head(200)

In [ ]:
count = ds_clean['tpl_id'].value_counts().sort_values(ascending=1)
count = pd.DataFrame(count)

count.columns = ["count"]

count=count[(count["count"] < 10)]
# print (count)
a = 0
temp = pd.DataFrame()
toAppend = pd.DataFrame()
# abc = dataset[(dataset['tpl_id'] == count.index.values[a])]
# print(count.index.values[a])
# print (abc)
for i in count["count"]:
    n = 0
    print (a,i , count.index.values[a])
    abc = ds_clean[(ds_clean['tpl_id'] == count.index.values[a])]
#     print (abc)
    for j in range (i,10):
        temp=abc.sample(n=1)
        toAppend=pd.concat([toAppend,temp],ignore_index=True)
    a+=1

In [ ]:
toAppend

In [ ]:
dataset=pd.concat([dataset,toAppend],ignore_index=True)

In [17]:
len(dataset['tpl_id'].value_counts())

171

In [ ]:
a=dataset.loc[22377]

In [28]:
dataset = dataset[pd.notnull(dataset['ticket_msg'])]
dataset = dataset[pd.notnull(dataset['ticket_title'])]
dataset = dataset[pd.notnull(dataset['tpl_id'])]
dataset = dataset[pd.notnull(dataset['ticket_category'])]

In [11]:
type(dataset)

pandas.core.frame.DataFrame

In [14]:
dataset.groupby['tpl_id']

TypeError: 'instancemethod' object has no attribute '__getitem__'

In [ ]:
datasetSorted = dataset.sort_values(["highest_tpl_id"],ascending=[1])

In [ ]:
dataset.ix[22378,:]

In [ ]:
datasetSorted2 = dataset.sort_values(["ticket_category"],ascending=[1])

In [ ]:
datasetSorted[['highest_tpl_id','tpl_n_ratio']].groupby(['highest_tpl_id']).agg(['count'])

In [ ]:
ds_X = datasetSorted.ix[:,["ticket_title","ticket_msg","ticket_category","highest_tpl_id"]]

In [29]:
ds_X = dataset

In [30]:
ds_y = dataset["tpl_id"]

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(ds_y)

In [ ]:
le.classes_

In [ ]:
a=le.transform(ds_y)

In [ ]:
pd.DataFrame(a)

In [32]:
XTrain,XTest,yTrain,yTest = ct.train_test_split(ds_X,ds_y,test_size=0.2,random_state=7)

In [ ]:
XTrain['tpl_id'].value_counts()

In [ ]:
XTrain.groupby(['highest_tpl_id']).agg(['count'])

In [31]:
pipe = ct.Pipeline([
                ("features", ct.FeatureUnion([
                            ("message", ct.Pipeline([
                                        ("select_text", ColumnSelector("ticket_msg")),       
                                        ("debug1", ct.PrintThat()),
                                        ("preprocessing_input", ApplyFunction(ct.preprocess_input)),
                                        ("delete_link", ApplyFunction(ct.clean_link)),
                                        ("clean_text", ApplyFunction(ct.clean_symbol)),
                                        ("hashing_vectorizer", ct.HashingVectorizer()),
                                    ])
                            ),
                            ("title", ct.Pipeline([
                                        ("select_text", ColumnSelector("ticket_title")),                                        
                                        ("preprocessing_input", ApplyFunction(ct.preprocess_input)),
                                        ("take_link", ApplyFunction(ct.take_link)),
                                        ("clean_link", ApplyFunction(ct.clean_symbol)),
                                        ("hashing_vectorizer", ct.HashingVectorizer()),
#                                         ("debug2", PrintThat())
                                    ])
                            ),
#                             ("category", ColumnSelector("ticket_category")),
 
                        ], transformer_weights={"message":1, "title":1})
                ),

                ("classifier", ct.LinearSVC(multi_class='ovr'))

            ])

In [34]:
a=pipe.decision_function(XTest)
# a=DataFrame(a)


15106    mengapa pembayaran saya belum di konfirmasi da...
2144     Mohon nama/domain toko saya diganti dari yang ...
12255    Halo, saya mau kasih saran saja supaya kedepan...
12807    Hallo, saya mau ubah toko saya menjadi Family ...
42441    Saya pesan Jas Hujan pada tgl 29 Maret lalu. S...
17121    Pesanan saya ditolak  dari toko  untuk  pengem...
14766    apakah pesanan si pembeli telah sampai? sampai...
5201     Tgl 7 maret 2016 saya transfer ke rekening Ban...
1671     Mohon dibantu, sdh terkonfirmasi pembayaran Rp...
7557     Saya penjual. Dalam fitur pencarian produk, ad...
43180    sdh bertransaksi senilai 105.357 menggunakan k...
158            Ingin merubah nama tokoh menjadi NutriSport
43369    Saya barusan belanja senilai Rp. 571.900,- mel...
10983    Saya pesan 'green laser pointer 1 mata' pada p...
46310    sudah lima hari tidak ada respon dan pengirima...
46827    sudah bayar utk 3 invoice, tapi sampai detik i...
16320    saya pengguna baru dan saya sudah mentransfer .

NotFittedError: This LinearSVC instance is not fitted yet

In [33]:
a=ColumnSelector("ticket_category")
XTrain[a]

KeyError: ColumnSelector(column='ticket_category')

In [ ]:
XTrain

In [ ]:
start_time = time.time()
print type(XTrain)
print type(yTrain)

pipe.fit_transform(XTrain, yTrain)

p_time = time.time() - start_time
print("predict time is {}".format(p_time))

In [ ]:
y_predict=pipe.predict(XTest)

In [ ]:
y_predict.decision_function(XTest)

In [ ]:
#     cross validation
start_time = time.time()
cv_score = ct.cross_val_score(pipe, XTrain, yTrain, cv=5)
cv_time = time.time() - start_time
print("predict time is {}".format(cv_time))

print("CV accuracy: %0.2f (+/- %0.2f)" % (cv_score.mean(), cv_score.std() * 2))